In [2]:
# !pip install setfit
# !pip install transformers==4.40.2
# !pip install optuna

This model remove "Entity:" and "Sentence:" in the input

In [3]:
import pandas as pd
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset
from sklearn.model_selection import train_test_split
from datasets import Dataset
import torch
import gc

In [4]:
#Load dataset
df = pd.read_csv('/content/manual_label - consolidated.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
# Check dataset
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,entity,sentence,paragraph,sentence_original,label,class_ID
0,retailer,which gives the supplier 's profit as and the ...,which gives the supplier's profit as and the r...,which gives the supplier's profit as and the r...,background_information,3
1,Proposition 1,which gives the supplier 's profit as and the ...,which gives the supplier's profit as and the r...,which gives the supplier's profit as and the r...,belongs_to_article,1
2,Supply-Chain,which gives the supplier 's profit as and the ...,which gives the supplier's profit as and the r...,which gives the supplier's profit as and the r...,background_information,3
3,Petri net,which can be equivalently represented with a P...,which can be equivalently represented with a P...,which can be equivalently represented with a P...,background_information,3
4,business processes,which are sent by business processes accounts ...,A simple example of a synthesis process for IA...,which are sent by business processes accounts ...,background_information,3


In [6]:
# Extract needed columns
df = df[['entity', 'sentence_original', 'class_ID']]
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,entity,sentence_original,class_ID
0,retailer,which gives the supplier's profit as and the r...,3
1,Proposition 1,which gives the supplier's profit as and the r...,1
2,Supply-Chain,which gives the supplier's profit as and the r...,3
3,Petri net,which can be equivalently represented with a P...,3
4,business processes,which are sent by business processes accounts ...,3


In [7]:
# Combine entity and sentence into one text column
df['text'] = df.apply(lambda row: f"{row['entity']}, {row['sentence_original']}", axis=1)

# Rename 'class_ID' to 'label' to match SetFit expectations
df = df.rename(columns={'class_ID': 'label'})

# Split the dataset into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert the DataFrame to a Dataset object
train_dataset = Dataset.from_pandas(train_df[['text', 'label']])
test_dataset = Dataset.from_pandas(test_df[['text', 'label']])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-7-874786d81c0a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df.apply(lambda row: f"{row['entity']}, {row['sentence_original']}", axis=1)


In [8]:
train_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,entity,sentence_original,label,text
38,cybercrime,whereas the magnitude of the coefficient of fi...,1,"cybercrime, whereas the magnitude of the coeff..."
143,framework,word2vec method is a self-supervised framework...,3,"framework, word2vec method is a self-supervise..."
84,organizational level,"yuqing Ren, Sara kiesler, and Susan R. Fussell...",2,"organizational level, yuqing Ren, Sara kiesler..."
55,online survey,"zoomeRaNG GeNeRated a RaNdom set of 1,200 sale...",1,"online survey, zoomeRaNG GeNeRated a RaNdom se..."
217,evaluation of Internet,whose primary goal is to produce theory driven...,3,"evaluation of Internet, whose primary goal is ..."
...,...,...,...,...
106,CFI was 0.924,"x 2 /df was 3.67, which was below the acceptab...",1,"CFI was 0.924, x 2 /df was 3.67, which was bel..."
14,individual-level,whether the predictors display curvilinear eff...,2,"individual-level, whether the predictors displ..."
92,information technology,"yet, as researchers, we do not often delve int...",3,"information technology, yet, as researchers, w..."
179,British,with other international airlines such as Qant...,3,"British, with other international airlines suc..."


In [9]:
train_dataset

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 202
})

In [10]:
test_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 51
})

In [11]:
import optuna
from typing import Dict, Union, Any
from setfit import SetFitModel, Trainer

from setfit import SetFitModel
from typing import Dict, Any

import torch

device = torch.device("cuda:0")

# Define the model initialization function
def model_init(params: Dict[str, Any]) -> SetFitModel:
    params = params or {}
    max_iter = params.get("max_iter", 100)
    solver = params.get("solver", "liblinear")

    # Use a dictionary to pass head parameters for the SetFit model
    head_params = {
        "max_iter": max_iter,
        "solver": solver,
    }

    return SetFitModel.from_pretrained(
        "sentence-transformers/paraphrase-mpnet-base-v2",
        head_params=head_params
    ).to(device)

# Define the hyperparameter space for tuning
def hp_space(trial: optuna.Trial) -> Dict[str, Union[float, int, str]]:
    return {
        "num_epochs": trial.suggest_int("num_epochs", 1, 2),
        "batch_size": trial.suggest_categorical("batch_size", [6]),
        "solver": trial.suggest_categorical("solver", ["newton-cg", "lbfgs", "liblinear"]),
    }

# Initialize the trainer
trainer = Trainer(
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    model_init=model_init,
)

# Perform hyperparameter search
best_run = trainer.hyperparameter_search(
    direction="maximize",
    hp_space=hp_space,
    n_trials=2
)

print("Best trial: ", best_run)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

...

***** Running evaluation *****


[I 2024-06-29 17:09:05,207] Trial 0 finished with value: 0.6862745098039216 and parameters: {'num_epochs': 1, 'batch_size': 6, 'solver': 'liblinear'}. Best is trial 0 with value: 0.6862745098039216.
Trial: {'num_epochs': 2, 'batch_size': 6, 'solver': 'newton-cg'}
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 25602
  Batch size = 6
  Num epochs = 2
  Total optimization steps = 8534


Step,Training Loss


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
***** Running evaluation *****
[I 2024-06-29 18:06:54,837] Trial 1 finished with value: 0.6862745098039216 and parameters: {'num_epochs': 2, 'batch_size': 6, 'solver': 'newton-cg'}. Best is trial 0 with value: 0.6862745098039216.


Best trial:  BestRun(run_id='0', objective=0.6862745098039216, hyperparameters={'num_epochs': 1, 'batch_size': 6, 'solver': 'liblinear'}, backend=<optuna.study.study.Study object at 0x7a899ce6ff70>)


In [12]:
trainer.apply_hyperparameters(best_run.hyperparameters, final_model=True)
trainer.train()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 25602
  Batch size = 6
  Num epochs = 1
  Total optimization steps = 4267


Step,Training Loss


In [13]:
trainer.model.save_pretrained('./optimized-model')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
# Evaluate the model on the test set
results = trainer.evaluate()
print(f"Evaluation results: {results}")

***** Running evaluation *****


Evaluation results: {'accuracy': 0.6862745098039216}


In [15]:
from sklearn.metrics import classification_report
from datasets import load_metric

# Load the metric
metric = load_metric("f1")

# Get predictions from the model
preds = model(test_dataset["text"])
labels = test_dataset["label"]

# Calculate F1 score
f1_score = metric.compute(predictions=preds, references=labels, average="weighted")
print(f"F1 score: {f1_score['f1']}")

# Define class names manually
class_names = ["1", "2", "3"]  # Replace with your actual class names

# Generate classification report
report = classification_report(labels, preds, target_names=class_names)
print(report)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-15-737161691913>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1")


The repository for f1 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/f1.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


NameError: name 'model' is not defined